In [ ]:

# ? check os name

import os
print(os.name)

# ? check operating system

import platform
print(platform.system())

# ? check operating system

import os
from unix_idle import unix_idle
from win_idle import win_idle

if os.name == 'nt':
    win_idle()
elif os.name == 'posix':
    unix_idle()

In [ ]:
# network.py

import pyshark

interface = 'wlp3s0'
output_file = 'captured_packets.pcap'
capture = pyshark.LiveCapture(interface=interface, output_file=output_file)

print(f"Capturing packets on interface: {interface}")
print(f"Saving captured packets to: {output_file}")

try:
    capture.sniff(timeout=20)
except KeyboardInterrupt:
    print("Stopping packet capture...")

print("Packet capture complete.")

In [ ]:
# .env
VT_API_KEY = "YOUR_API_KEY_FROM_VIRUSTOTAL"

In [ ]:
# config.yml

exclude_dirs:
  - /home/aditya/Documents/GitHub/defacement/stuff

include_dirs:
  - /home/aditya/Documents/GitHub/mawscan

rules:
  - /home/aditya/Documents/GitHub/mawscan/rules/md5sum

md5_hash:
  - /home/aditya/Documents/GitHub/mawscan/rules/md5sum

sha1_hash:
  - /home/aditya/Documents/GitHub/mawscan/rules/sha1sum

sha256_hash:
  - /home/aditya/Documents/GitHub/mawscan/rules/sha256sum

sha512_hash:
  - /home/aditya/Documents/GitHub/mawscan/rules/sha512sum

In [ ]:
# VIRUSTOTAL FILE SCANNER
# vt_file_scanner.py

import vt
from dotenv import get_key, load_dotenv

load_dotenv()
api_key = get_key('.env', "VT_API_KEY")
client = vt.Client(api_key)

with open(".env", "rb") as f:
    analysis = client.scan_file(f, wait_for_completion=True)

print(list(analysis.results.keys()))

'''
{
'Bkav': {'method': 'blacklist', 'engine_name': 'Bkav', 'engine_version': '2.0.0.1', 'engine_update': '20240509', 'category': 'undetected', 'result': None}, 
'Lionic': {'method': 'blacklist', 'engine_name': 'Lionic', 'engine_version': '7.5', 'engine_update': '20240509', 'category': 'undetected', 'result': None}, 
}
'''

In [ ]:
# VIRUSTOTAL HASH SCANNER
# vt_hash_scanner.py

from dotenv import get_key, load_dotenv
import vt

load_dotenv()
api_key = get_key('.env', "VT_API_KEY")
client = vt.Client(api_key)

FILE_HASH = "b2dd7f9bb24428b0e2ed30b9373fe033d981a29415576b4c654c0d999dd109e5"

analysis = client.get_object("/files/{}", FILE_HASH)
print(analysis.last_analysis_stats)

In [ ]:
# MEMORY DUMP ANALYSIS SCRIPT
# mem.py

import os
from utils.nix_mem import create_memory_dump_linux
from utils.win_mem import create_memory_dump_windows

if __name__ == "__main__":

    target_process_id = 1234
    output_file = "memory_dump.dmp"

    if os.name == 'nt':
        create_memory_dump_windows(target_process_id, output_file)
    elif os.name == 'posix':
        create_memory_dump_linux(target_process_id, output_file)
    else:
        print("Unsupported operating system.")


In [ ]:
# LINUX MEMORY DUMP
# utils / nix_mem.py

import subprocess

def create_memory_dump_linux(process_id, output_file):
    
    try:
        result = subprocess.run(['gdb', '--batch', '-p', str(process_id), '-ex', f'dump memory {output_file}'], capture_output=True)

        if result.returncode == 0:
            print("Memory dump created successfully.")
            return True
        else:
            print("Failed to create memory dump.")
            print(result.stderr.decode())
            return False

    except Exception as e:
        print(f"Error creating memory dump: {e}")
        return False

In [ ]:
# WINDOWS MEMORY DUMP
# utils / win_mem.py

import ctypes

def create_memory_dump_windows(process_id, output_file):
    
    try:
        kernel32 = ctypes.WinDLL('kernel32')

        hProcess = kernel32.OpenProcess(0x1F0FFF, False, process_id)
        if hProcess == 0:
            print("Failed to open process.")
            return False

        hFile = kernel32.CreateFileW(output_file, 0x10000000, 0, None, 2, 0x80, None)
        if hFile == -1:
            print("Failed to create dump file.")
            return False

        result = kernel32.MiniDumpWriteDump(hProcess, process_id, hFile, 2, None, None, None)

        kernel32.CloseHandle(hFile)
        kernel32.CloseHandle(hProcess)

        if result:
            print("Memory dump created successfully.")
            return True
        else:
            print("Failed to create memory dump.")
            return False

    except Exception as e:
        print(f"Error creating memory dump: {e}")
        return False

In [ ]:
import requests

load_dotenv()

api_key = get_key('.env', "VT_API_KEY")

url = "https://www.virustotal.com/api/v3/files/630325cac09ac3fab908f903e3b00d0dadd5fdaa0875ed8496fcbb97a558d0da"

headers = {"accept": "application/json", "x-apikey": api_key}

response = requests.get(url, headers=headers)

val = response.json()
t = response.text

print(t["data"]["id"])
print(val["data"]["id"])

In [ ]:
import vt
from dotenv import get_key, load_dotenv

load_dotenv()
api_key = get_key('.env', "VT_API_KEY")
client = vt.Client(api_key)

with open(".env", "rb") as f:
    analysis = client.scan_file(f, wait_for_completion=True)

print(analysis.results)

print(list(analysis.results.keys()))